# Features to replace crs_arr_time and crs_dep_time

In [19]:
import sys
sys.path.append("../..")  # <-  This should point to the root directory of the project relative to this file

from custom_scripts import preprocessing

import math
import numpy as np
import pandas as pd

In [2]:
raw_test_flights = preprocessing.get_test_flights()
average_delays = pd.read_csv('../../data/preprocessing/averages_by_fl_num.csv')


In [21]:
test_flights = raw_test_flights.copy()
print(test_flights.shape)
test_flights.head(1)

(150623, 9)


,fl_date,op_unique_carrier,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2020-01-01,WN,5888,13891,14771,1810,1945,95,363


In [30]:
### pad values
test_flights['arrival_time'] = test_flights['crs_arr_time'].astype(str).str.zfill(4)
test_flights['departure_time'] = test_flights['crs_dep_time'].astype(str).str.zfill(4)
### convert to datetime
test_flights['arrival_time'] = pd.to_datetime(test_flights['arrival_time'], format = '%H%M', errors='coerce')
test_flights['departure_time'] = pd.to_datetime(test_flights['departure_time'], format = '%H%M', errors='coerce')
### convert to minute of day
test_flights['arrival_time'] = test_flights['arrival_time'].dt.hour * 60 + test_flights['arrival_time'].dt.minute
test_flights['departure_time'] = test_flights['departure_time'].dt.hour * 60 + test_flights['departure_time'].dt.minute
### normalize with 2*PI
test_flights['arrival_time'] = 2 * math.pi * test_flights['arrival_time'] / test_flights['arrival_time'].max()
test_flights['departure_time'] = 2 * math.pi * test_flights['departure_time'] / test_flights['departure_time'].max()

In [33]:
test_flights['arr_time_sin'] = np.sin(test_flights['arrival_time'])
test_flights['arr_time_cos'] = np.cos(test_flights['arrival_time'])

test_flights['dep_time_sin'] = np.sin(test_flights['departure_time'])
test_flights['dep_time_cos'] = np.cos(test_flights['departure_time'])

test_flights.drop(['arrival_time','departure_time'],1,inplace=True)

In [34]:

test_flights


,fl_date,op_unique_carrier,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_time_sin,arr_time_cos,dep_time_sin,dep_time_cos
0,2020-01-01,WN,5888,13891,14771,1810,1945,95,363,-0.895278,0.445508,-0.998899,0.046921
1,2020-01-01,WN,6276,13891,14771,1150,1320,90,363,-0.344299,-0.938860,0.041468,-0.999140
2,2020-01-01,WN,4598,13891,14831,2020,2130,70,333,-0.605654,0.795728,-0.817025,0.576603
3,2020-01-01,WN,4761,13891,14831,1340,1455,75,333,-0.693471,-0.720485,-0.424870,-0.905254
4,2020-01-01,WN,5162,13891,14831,915,1035,80,333,0.360643,-0.932704,0.658080,-0.752948
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150618,2020-01-07,9E,4813,11433,12478,1755,1952,117,509,-0.881245,0.472660,-0.999828,-0.018556
150619,2020-01-07,9E,4814,11996,12953,600,759,119,610,0.867477,-0.497477,0.999999,-0.001092
150620,2020-01-07,9E,4815,10397,15919,1715,1816,121,589,-0.997327,0.073071,-0.981393,-0.192011
150621,2020-01-07,9E,4815,15919,10397,1851,2145,114,589,-0.552276,0.833661,-0.974580,0.224041
